In [ ]:
import torch
import torch.nn as nn

In [1]:
def printmb(obj):
    print(f'{obj / 1024 / 1024} MB')

def printm():
    printmb(torch.cuda.memory_allocated())
    printmb(torch.cuda.memory_reserved())

In [ ]:
model = nn.Linear(1024, 1024, bias=False).cuda()
printm()

In [ ]:
inputs = torch.tensor([1.0]*1024).cuda()  # shape = (1024)  memory + 4096
outputs = model(inputs)
printm()

In [ ]:
loss = sum(outputs)  # memory + 512(torch allocate分配最小单位)
temp = torch.cuda.memory_allocated()
loss.backward()
printmb(torch.cuda.memory_allocated() - temp)  # 第一次增加4194304

In [2]:
import torch


# 模型初始化
linear1 = torch.nn.Linear(1024, 1024, bias=False).cuda()  # + 4194304
printmb(torch.cuda.memory_allocated())
linear2 = torch.nn.Linear(1024, 1, bias=False).cuda()  # + 4096
printmb(torch.cuda.memory_allocated())

# 输入定义
# shape = (1024,1024) # + 4194304
inputs = torch.tensor([[1.0]*1024]*1024).cuda()
printmb(torch.cuda.memory_allocated())

# 前向传播
loss = sum(linear2(linear1(inputs)))  # shape = (1) # memory + 4194304 + 512
printmb(torch.cuda.memory_allocated())

# 后向传播
loss.backward()  # memory - 4194304 + 4194304 + 4096
printmb(torch.cuda.memory_allocated())

# 再来一次~
# shape = (1) # memory + 4194304  (512没了，因为loss的ref还在)
loss = sum(linear2(linear1(inputs)))
printmb(torch.cuda.memory_allocated())
loss.backward()  # memory - 4194304
printmb(torch.cuda.memory_allocated())

4.0 MB
4.00390625 MB
8.00390625 MB
12.00439453125 MB
12.00830078125 MB
16.00830078125 MB
12.00830078125 MB
